# Project Title
### Data Engineering Capstone Project

#### Project Summary
The goal of this project is to create an ETL pipeline using I94 immigration and city temperature data to create a database optimized for queries on immigration events. This database can be used to answer questions that are related to immigration behavior.

The project follows these steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

Please note that this notebook is designed to be run on Udacity's notebook workspace as the data is provided internally by Udacity.  Also note that we will be using Pandas Redshift as a convenience method to upload our Pandas dataframes as tables to Amazon Redshift.

In [3]:
!pip install pandas_redshift

    100% |████████████████████████████████| 3.0MB 7.9MB/s eta 0:00:01    41% |█████████████▏                  | 1.2MB 11.0MB/s eta 0:00:01    65% |█████████████████████           | 1.9MB 17.4MB/s eta 0:00:01    94% |██████████████████████████████▎ | 2.8MB 19.2MB/s eta 0:00:01


In [4]:
# Do all imports and installs here
import pandas as pd
import pandas_redshift as pr
import datetime
import re

### Step 1: Scope the Project and Gather Data

#### Scope 

The intent of this project is to build a data warehouse on AWS Redshift and I plan on designing an ETL pipeline to load data into tables for OLAP.  The data that will be folded in will include sources from the I94 US Immigration dataaset provided by Udacity, as well as data that involves airport codes, US city demographics and data regarding temperature.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The I94 immigration [data](https://travel.trade.gov/research/reports/i94/historical/2016.html) origtinates from the US National Tourism and Trade Office.  An additional intricacy is that is provided in SAS7BDAT [format](https://cran.r-project.org/web/packages/sas7bdat/vignettes/sas7bdat.pdf) and is a binary database storage format.  It contains international visitor arrival statistics by world regions and select countries (including top 20), type of visa, mode of transportation, age groups, states visited (first intended address only), and the top ports of entry (for select countries).  There are 12 parts of this datasets with over 40 million records combined.  However we will concentrate on the data that comes from April 2016 that consists of roughly 3 million records.  Some relevant attributes include:

* `i94yr` - 4 digit year
* `i94mon` - Numeric month
* `i94cit` - 3 digit code of origin city
* `i94port` - 3 character code of destination USA city
* `arrdate` - Arrival date in the USA
* `i94mode` - 1 digit travel code
* `depdate` - Departure date from the USA
* `i94visa` - Reason for immigration

In [5]:
# Read in the data here - This takes some time to run.  Be patient.
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [8]:
# Show the first 5 elements of this dataframe
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


#### Relevant Column Names to be used in our analysis
Column Name|Description
---|---
*cicid*|ID that uniquely identify one record in the dataset
i94yr | 4 digit year
i94mon | Numeric month
i94cit | 3 digit code of source city for immigration (Birth country)
i94res | 3 digit code of source country for immigration (Residence country)
i94port | Port admitted through
arrdate | Arrival date in the USA
i94mode | Mode of transportation (1 = Air, 2 = Sea, 3 = Land, 9 = Not reported)
i94addr | State of arrival
depdate | Departure date
i94bir | Age (years) of respondent
i94visa | Visa codes broken down into 3 categories: (1 = Business; 2 = Pleasure; 3 = Student)
count | Used for summary statistics
dtadfile | Character Date Field
visapost | Department of State where where Visa was issued
occup | Occupation to be performed in U.S.
entdepa | Arrival Flag - Admitted or parolled into the US
entdepd | Departure Flag - Departed, lost visa, or deceased
entdepu | Update Flag - Update of visa, either apprehended, overstayed, or updated to permanent resident
matflag | Match flag
biryear | 4 digit year of birth
dtaddto | Character date field to when admitted to the US
gender | Gender
insnum | INS number
airline | Airline used to arrive in U.S.
admnum | Admission number - Should be unique and not null
fltno | Flight number of the airline used to arrive in the US
visatype | Class of admission legally allowing the non-immigrant to temporarily stay in US

##### US City Demographic Data

[This dataset](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) contains various information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000.  This comes from the 2015 American Community Survey from the US Census Bureau's 2015 American Community Survey.

In [9]:
# Read in the US city demographics data and show the first 5 rows of the dataframe
us_city_demographics_df = pd.read_csv('us-cities-demographics.csv', delimiter=';')
us_city_demographics_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


### Step 2: Explore and Assess the Data
#### Explore the Data 


*I94 Immigration Data*

The following columns have missing data:

* `i94mode`
* `i94bir`
* `dtadfile`
* `visapost`
* `occup`
* `entdepa`
* `entdepd`
* `entdepu`
* `matflag`
* `biryear`
* `dtaddto`
* `gender`
* `insnum`
* `airline`
* `fltno`

Due to this being missing, we will drop these columns.  

The most important columns we will keep are:

* `cicid` (naturally)
* `i94yr`
* `i94mon`,
* `i94cit`
* `i94res`
* `i94port`
* `arrdate`
* `i94addr`
* `depdate`
* `i94visa`
* `count`
* `admnum`
* `visatype`

Let's display the total number of records for each column below.  Take note that we are dealing with 3,096,313 total records.  Any columns that don't have this amount are deemed missing and are removed from further analysis.

In [10]:
# Show non-null counts for each column of the immigration data
df.count()

cicid       3096313
i94yr       3096313
i94mon      3096313
i94cit      3096313
i94res      3096313
i94port     3096313
arrdate     3096313
i94mode     3096074
i94addr     2943941
depdate     2953856
i94bir      3095511
i94visa     3096313
count       3096313
dtadfile    3096312
visapost    1215063
occup          8126
entdepa     3096075
entdepd     2957884
entdepu         392
matflag     2957884
biryear     3095511
dtaddto     3095836
gender      2682044
insnum       113708
airline     3012686
admnum      3096313
fltno       3076764
visatype    3096313
dtype: int64

*US City Demographic Data*

The following columns have missing data:

* `Male Population`
* `Female Population`
* `Number of Veterans`
* `Foreign-born`
* `Average Household Size` 

We only interested in the `Total Population` in any case.  Take note that there are 2891 records for this dataset so any columns that don't match this should be removed.

In [11]:
# Show the non-null counts for each column in the demographics data
us_city_demographics_df.count()

City                      2891
State                     2891
Median Age                2891
Male Population           2888
Female Population         2888
Total Population          2891
Number of Veterans        2878
Foreign-born              2878
Average Household Size    2875
State Code                2891
Race                      2891
Count                     2891
dtype: int64

#### Cleaning Steps

#### I94 Immigration Data
As noted above, we will select out the relevant columns and remove the columns that are missing.

In [12]:
# Select out the relevant columns and drop any rows that are missing
us_immigration_df = df[['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate', 'i94addr', 'depdate', 'i94visa', 'count', 'admnum', 'visatype']]
us_immigration_df = us_immigration_df.dropna()

# There is an escape character with the i94addr column
# Remove this using regular expressions where any repeated whitespace is removed
us_immigration_df['i94addr'] = us_immigration_df['i94addr'].map(lambda col: re.sub('\W+', '', col))

In [13]:
# Show the first 5 elements of this new dataframe
us_immigration_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94addr,depdate,i94visa,count,admnum,visatype
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,MI,20691.0,2.0,1.0,6.666432e+08,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,MA,20567.0,2.0,1.0,9.246846e+10,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,MA,20567.0,2.0,1.0,9.246846e+10,B2
5,18.0,2016.0,4.0,101.0,101.0,NYC,20545.0,MI,20555.0,1.0,1.0,9.247104e+10,B1
6,19.0,2016.0,4.0,101.0,101.0,NYC,20545.0,NJ,20558.0,2.0,1.0,9.247140e+10,B2


#### US City Demographic Data
As noted above, we select out the relevant columns.  Also note that we should drop duplicates!

In [14]:
# Select out the relevant columns and drop any duplicates
us_city_demographics_df = us_city_demographics_df[['State Code', 'City', 'State', 'Total Population']].drop_duplicates()

# Show the first 5 elements of this new dataframe
us_city_demographics_df.head()

,State Code,City,State,Total Population
0,MD,Silver Spring,Maryland,82463
1,MA,Quincy,Massachusetts,93629
2,AL,Hoover,Alabama,84839
3,CA,Rancho Cucamonga,California,175232
4,NJ,Newark,New Jersey,281913


### Step 3: Define the Data Model

#### 3.1 Conceptual Data Model

As we naturally learned in this course, we will use a more denormalized version of this combined data to minimize the amount of joins and to allow our data model to be used and interpreted in a more meaningful way.  Specifically, I chose to use the star schema with the immigration data serving as the fact table while the data that deals with dates and the states themselves are the dimension tables.  This will ultimately allow us to aggregate information about immigration based on fine grained time units, such as month or year as well the state itself.

#### Fact Table

`immigration`

- Columns: *`cicid`*, `i94yr`, `i94mon`, `i94cit`, `i94res`, `i94port`, `arrdate`, `i94addr`, `depdate`, `i94visa`, `count`, `admnum`, `visatype`

`cicid` will serve as the primary key

#### Dimension Tables

`date` - arrival and departure date in immigration broken down into specific units 
- *`sas_date`*, date, day, month, year, weekday

`sas_date` will serve as the primary key.  This is a custom column that will be created here.  The SAS date is a value that represents the number of days between January 1, 1960, and a specified date.

`state` - total population of the states
- *`state_code`*, state, total_population

`state_code` will serve as the primary key.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

This will be outlined below.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### Create the state table

In [15]:
# We group all city population by state and aggregate by summing over them
state_pop = us_city_demographics_df.groupby('State').sum().reset_index()
state_pop.head()

,State,Total Population
0,Alabama,1049629
1,Alaska,298695
2,Arizona,4499542
3,Arkansas,589879
4,California,24822460


In [16]:
# Perform a right join to merge the state code and state together so we establish a mappin
# between state code, the full name of the state as well as its population.
state_pop_df = pd.merge(us_city_demographics_df[['State Code', 'State']].drop_duplicates(), state_pop, how='right', on='State')
state_pop_df.columns = ['state_code', 'state', 'population']

# Note we will rename these columns so that we can write to Redshift later for easy access
state_pop_df.head()

,state_code,state,population
0,MD,Maryland,1312129
1,MA,Massachusetts,2015457
2,AL,Alabama,1049629
3,CA,California,24822460
4,NJ,New Jersey,1428908


#### Create the date table

For the SAS date, we will convert this into a readable date format that will allow us to do datetime processing with ease.  We'll do the same foro the departure date as well.

We extract all values from arrdate and depdate and convert them into readable date format.

We will first get the arrival date as a table in readable form and decompose the date into further units such as weekday, day, month and year.

In [17]:
# Make a copy of the arrdate column for us to manipulate
arrdate = us_immigration_df[['arrdate']].copy()
# Rename the arrdate column to sas_date for final storage on Redshift
arrdate = arrdate.rename(columns={"arrdate": "sas_date"})

# Make this new column so that it provides date in readable format
arrdate['date'] = arrdate['sas_date'].apply(lambda col: datetime.date(1960, 1, 1) + datetime.timedelta(days=col))

# Also put in the weekday, day, month and year as new columns
arrdate['weekday'] = arrdate['date'].apply(lambda col: col.weekday())
arrdate['day'] = arrdate['date'].apply(lambda col: col.day)
arrdate['month'] = arrdate['date'].apply(lambda col: col.month)
arrdate['year'] = arrdate['date'].apply(lambda col: col.year)

# Drop duplicates
arrdate = arrdate.drop_duplicates()

In [18]:
print(arrdate.shape)
arrdate.head()

(30, 6)


,sas_date,date,weekday,day,month,year
2,20545.0,2016-04-01,4,1,4,2016
101084,20546.0,2016-04-02,5,2,4,2016
197740,20547.0,2016-04-03,6,3,4,2016
290895,20548.0,2016-04-04,0,4,4,2016
384303,20549.0,2016-04-05,1,5,4,2016


Do the same for the depdate

In [19]:
depdate = us_immigration_df[['depdate']].copy()
# Rename the arrdate column to sas_date for final storage on Redshift
depdate = depdate.rename(columns={"depdate": "sas_date"})

# Make new column so that it provides date in readable format
depdate['date'] = depdate['sas_date'].apply(lambda col: datetime.date(1960, 1, 1) + datetime.timedelta(days=col))

# Also put in the weekday, day, month and year as new columns
depdate['weekday'] = depdate['date'].apply(lambda col: col.weekday())
depdate['day'] = depdate['date'].apply(lambda col: col.day)
depdate['month'] = depdate['date'].apply(lambda col: col.month)
depdate['year'] = depdate['date'].apply(lambda col: col.year)

# Drop duplicates
depdate = depdate.drop_duplicates()

In [20]:
print(depdate.shape)
depdate.head()

(218, 6)


,sas_date,date,weekday,day,month,year
2,20691.0,2016-08-25,3,25,8,2016
3,20567.0,2016-04-23,5,23,4,2016
5,20555.0,2016-04-11,0,11,4,2016
6,20558.0,2016-04-14,3,14,4,2016
8,20553.0,2016-04-09,5,9,4,2016


Now that we have both arrival and departure dates, we should combine these into one final table as both the arrival and departure dates are both events we are interested in.  Simply do an outer join to help us accomplish this.

In [21]:
date = pd.merge(arrdate, depdate, how='outer')

In [57]:
print(date.shape)
date.head()

(218, 6)


,sas_date,date,weekday,day,month,year
0,20545.0,2016-04-01,4,1,4,2016
1,20546.0,2016-04-02,5,2,4,2016
2,20547.0,2016-04-03,6,3,4,2016
3,20548.0,2016-04-04,0,4,4,2016
4,20549.0,2016-04-05,1,5,4,2016


#### Moving the data onto Redshift

Now that we've prepared our data seen above, let's move it onto Amazon Redshift.  Take note that this will use a configuration file that requires information that is sensitive, so on submission these are nulled out.

In [23]:
import configparser

In [24]:
# Read in attributes from custom config file to access S3 and Redshift
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [25]:
# Attributes for accessing Redshift cluster
DWH_DB_USER = config.get("CLUSTER", "DB_USER")
DWH_DB_PASSWORD = config.get("CLUSTER", "DB_PASSWORD")
DWH_ENDPOINT = config.get("CLUSTER", "HOST")
DWH_PORT = config.get("CLUSTER", "DB_PORT")
DWH_DB = config.get("CLUSTER", "DB_NAME")

In [26]:
# Attributes for S3 access
AWS_ACCESS_KEY = config.get("AWS", "AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = config.get("AWS", "AWS_SECRET_ACCESS_KEY_ID")
AWS_BUCKET = config.get("AWS", "AWS_BUCKET")
AWS_BUCKET_SUBDIR = config.get("AWS", "AWS_BUCKET_SUBDIR")

In [27]:
# Connect to Redshift cluster
pr.connect_to_redshift(dbname=DWH_DB, host=DWH_ENDPOINT, port=DWH_PORT, user=DWH_DB_USER, password=DWH_DB_PASSWORD)

In [28]:
# Connect to S3
pr.connect_to_s3(aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_ACCESS_KEY, bucket=AWS_BUCKET, subdirectory=AWS_BUCKET_SUBDIR)

`pandas_redshift.pandas_to_redshift` is a convenience method that will place our dataframe in an intermediate CSV form on a S3 bucket of our choice, then it will create a new table on Redshift based on this intermediate form.  Think of this as creating a staging table then final table in one sweep.  We will now place our tables on S3, then Redshift.

In [52]:
# Create US immigration fact table - Put on S3 then go to Redshift
pr.pandas_to_redshift(data_frame=us_immigration_df, redshift_table_name = 'public.immigration', index=False)

Pandas Redshift | 2021-01-01 16:26:39,130 | pandas_redshift.core | INFO | saved file public.immigration-ee3aa726-d442-4a89-85dc-83a24e2032f4.csv in bucket capstone_data/public.immigration-ee3aa726-d442-4a89-85dc-83a24e2032f4.csv
Pandas Redshift | 2021-01-01 16:26:39,144 | pandas_redshift.core | INFO | create table public.immigration (cicid REAL, i94yr REAL, i94mon REAL, i94cit REAL, i94res REAL, i94port VARCHAR(256), arrdate REAL, i94addr VARCHAR(256), depdate REAL, i94visa REAL, count REAL, admnum REAL, visatype VARCHAR(256)) diststyle even
Pandas Redshift | 2021-01-01 16:26:39,148 | pandas_redshift.core | INFO | CREATING A TABLE IN REDSHIFT
Pandas Redshift | 2021-01-01 16:26:39,390 | pandas_redshift.core | INFO | 
    copy public.immigration
    from 's3://rayryeng-dend-capstone/capstone_data/public.immigration-ee3aa726-d442-4a89-85dc-83a24e2032f4.csv'
    delimiter ','
    ignoreheader 1
    csv quote as '"'
    dateformat 'auto'
    timeformat 'auto'
    
        access_key_id '***

In [54]:
# Create state dimension table - Put on S3 then go to Redshift
pr.pandas_to_redshift(data_frame=state_pop_df, redshift_table_name = 'public.state')

Pandas Redshift | 2021-01-01 16:30:15,812 | pandas_redshift.core | INFO | saved file public.state-c4560e47-ab34-4ad5-8d1d-9ea9353f8c5f.csv in bucket capstone_data/public.state-c4560e47-ab34-4ad5-8d1d-9ea9353f8c5f.csv
Pandas Redshift | 2021-01-01 16:30:15,816 | pandas_redshift.core | INFO | create table public.state (state_code VARCHAR(256), state VARCHAR(256), population BIGINT) diststyle even
Pandas Redshift | 2021-01-01 16:30:15,820 | pandas_redshift.core | INFO | CREATING A TABLE IN REDSHIFT
Pandas Redshift | 2021-01-01 16:30:16,063 | pandas_redshift.core | INFO | 
    copy public.state
    from 's3://rayryeng-dend-capstone/capstone_data/public.state-c4560e47-ab34-4ad5-8d1d-9ea9353f8c5f.csv'
    delimiter ','
    ignoreheader 1
    csv quote as '"'
    dateformat 'auto'
    timeformat 'auto'
    
        access_key_id '********'
        secret_access_key '********'
        
    
    ;
Pandas Redshift | 2021-01-01 16:30:16,066 | pandas_redshift.core | INFO | FILLING THE TABLE IN REDS

In [55]:
# Create date dimension table - Put on S3 then go to Redshift
pr.pandas_to_redshift(data_frame=date, redshift_table_name = 'public.date')

Pandas Redshift | 2021-01-01 16:31:10,697 | pandas_redshift.core | INFO | saved file public.date-08be7eba-3d8c-4f3c-961d-8c70ff5ebf77.csv in bucket capstone_data/public.date-08be7eba-3d8c-4f3c-961d-8c70ff5ebf77.csv
Pandas Redshift | 2021-01-01 16:31:10,702 | pandas_redshift.core | INFO | create table public.date (sas_date REAL, date VARCHAR(256), weekday BIGINT, day BIGINT, month BIGINT, year BIGINT) diststyle even
Pandas Redshift | 2021-01-01 16:31:10,706 | pandas_redshift.core | INFO | CREATING A TABLE IN REDSHIFT
Pandas Redshift | 2021-01-01 16:31:10,949 | pandas_redshift.core | INFO | 
    copy public.date
    from 's3://rayryeng-dend-capstone/capstone_data/public.date-08be7eba-3d8c-4f3c-961d-8c70ff5ebf77.csv'
    delimiter ','
    ignoreheader 1
    csv quote as '"'
    dateformat 'auto'
    timeformat 'auto'
    
        access_key_id '********'
        secret_access_key '********'
        
    
    ;
Pandas Redshift | 2021-01-01 16:31:10,953 | pandas_redshift.core | INFO | FILLI

#### 4.2 Data Quality Checks
We will perform integrity checks to ensure that our data is in a sane format.  This will include some sample queries made directly to the tables as well as checking the number of records for each table.  `pandas_redshift.redshift_to_pandas` is a nice function that allows us to provide a SQL query as a string and it returns a Pandas dataframe of the resulting query.

##### 4.2.1 Number of rows should be non-zero
Check to see if we have any tables that are empty

In [58]:
def data_quality_check(table):
    """
    Checks to see if the total number of records is non-zero for a table
    
    Inputs:
        table - String to denote our table to check
    
    Returns:
        True/False to signify if number of records is non-zero
    """
    return len(pr.redshift_to_pandas(f"SELECT COUNT(*) FROM {table}")) > 0

In [59]:
for table in ['immigration', 'state', 'date']:
    if not data_quality_check(table):
        print(f"Data quality check failed. {table} returned no results.")
    else:
        print(f"Data quality check passed for {table}")

Data quality check passed for immigration
Data quality check passed for state
Data quality check passed for date


#### 4.2.2 Check to see if there are any null primary keys

Let's also check to see if there are any null records for the primary keys for the columns.

In [64]:
def data_quality_check_null(table, column):
    """
    Checks to see if there are any null records in a column
    
    Inputs:
        table - String to denote our table to check
        column - Column to check
    
    Returns:
        True/False to signify if there are null records in a column
    """
    return len(pr.redshift_to_pandas(f"SELECT COUNT(*) FROM {table} WHERE {column} IS NULL")) == 0

In [65]:
for table, column in zip(['immigration', 'state', 'date'], ['cicid', 'state_code', 'sas_date']):
    if data_quality_check_null(table, column):
        print(f"Data quality check failed. {table} has null records for column {column}")
    else:
        print(f"Data quality check passed for {table}, column {column}")

Data quality check passed for immigration, column cicid
Data quality check passed for state, column state_code
Data quality check passed for date, column sas_date


#### 4.2.3 Check the first 20 records of our final tables

Let's also do some simple checks on the actual contents.  Cycle through each table, grab 20 records and show them.

In [66]:
results = {table: pr.redshift_to_pandas(f"SELECT * FROM {table} LIMIT 20") for table in ['immigration', 'state', 'date']}

In [67]:
results['immigration']

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94addr,depdate,i94visa,count,admnum,visatype
0,18.0,2016.0,4.0,101.0,101.0,NYC,20545.0,MI,20555.0,1.0,1.0,9.247100e+10,B1
1,28.0,2016.0,4.0,101.0,101.0,ATL,20545.0,MA,20549.0,1.0,1.0,9.247890e+10,B1
2,39.0,2016.0,4.0,101.0,101.0,MIA,20545.0,FL,20574.0,2.0,1.0,9.250770e+10,B2
3,51.0,2016.0,4.0,101.0,117.0,MIA,20545.0,FL,20555.0,2.0,1.0,9.248630e+10,B2
4,69.0,2016.0,4.0,103.0,103.0,ATL,20545.0,FL,20560.0,2.0,1.0,5.543190e+10,WT
5,78.0,2016.0,4.0,103.0,103.0,BOS,20545.0,MA,20548.0,2.0,1.0,5.543370e+10,WT
6,86.0,2016.0,4.0,103.0,103.0,BOS,20545.0,NH,20550.0,1.0,1.0,5.545320e+10,WB
7,94.0,2016.0,4.0,103.0,103.0,CLT,20545.0,SC,20546.0,2.0,1.0,5.544580e+10,WT
8,105.0,2016.0,4.0,103.0,103.0,DET,20545.0,MI,20556.0,2.0,1.0,5.542590e+10,WT
9,116.0,2016.0,4.0,103.0,103.0,NEW,20545.0,CA,20567.0,2.0,1.0,5.542250e+10,WT


In [68]:
results['state']

,state_code,state,population
0,MA,Massachusetts,2015457
1,PA,Pennsylvania,2330856
2,CT,Connecticut,885581
3,GA,Georgia,1711032
4,PR,Puerto Rico,990226
5,ID,Idaho,398883
6,MO,Missouri,1519194
7,CO,Colorado,2935669
8,WA,Washington,2500107
9,IA,Iowa,733811


In [69]:
results['date']

,sas_date,date,weekday,day,month,year
0,20549.0,2016-04-05,1,5,4,2016
1,20557.0,2016-04-13,2,13,4,2016
2,20565.0,2016-04-21,3,21,4,2016
3,20573.0,2016-04-29,4,29,4,2016
4,20665.0,2016-07-30,5,30,7,2016
5,20610.0,2016-06-05,6,5,6,2016
6,20677.0,2016-08-11,3,11,8,2016
7,20630.0,2016-06-25,5,25,6,2016
8,20645.0,2016-07-10,6,10,7,2016
9,20588.0,2016-05-14,5,14,5,2016


In [70]:
# Finally close the cursor, commit and close the database connection, and remove variables from the environment.
pr.close_up_shop()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### Data Dictionary
#### Immigration

Column Name | Description
--- | ---
*cicid*|ID that uniquely identify one record in the dataset
i94yr | 4 digit year
i94mon | Numeric month
i94cit | 3 digit code of source city for immigration (Birth country)
i94res | 3 digit code of source country for immigration (Residence country)
i94port | Port admitted through
arrdate | Arrival date in the USA.  This is the SAS date which is the number of days since January 1, 1960 that the subject arrived.
i94addr | State of arrival
depdate | Departure date
i94visa | Visa codes broken down into 3 categories: (1 = Business; 2 = Pleasure; 3 = Student)
count | Used for summary statistics
admnum | Admission number - Should be unique and not null
visatype | Class of admission legally allowing the non-immigrant to temporarily stay in US

#### State

Column Name | Description
--- | ---
*`state_code`* | Two letter state code representing the state
`state` | Full name of the state
`population`| Population of the state

#### Date
Column Name | Description
--- | ---
*`sas_date`* | The number of days since January 1, 1960 that the subject arrived
`date` | Date in YYYY-MM-DD format
`weekday` | If the day fell onto a weekday.  (1 = Monday, 2 = Tuesday, 3 = Wednesday, 4 = Thursday, 5 = Friday, 0 = Weekend)
`day` | Code for which day of the week it was (1 = Monday, 2 = Tuesday, 3 = Wednesday, 4 = Thursday, 5 = Friday, 6 = Saturday, 7 = Sunday)
`month` | Code for which month of the week it was (1 = January, 2 = February, 3 = March, 4 = April, 5 = May, 6 = June, 7 = July, 8 = August, 9 = September, 10 = October, 11 = November, 12 = December)
`year` | Year

### Primary Keys

* `immigration`: `cicid`
* `state`: `state_code`
* `date`: `sas_date`


#### Step 5: Complete Project Write Up

* Clearly state the rationale for the choice of tools and technologies for the project.

We used a convenience library called [pandas-redshift](https://github.com/agawronski/pandas_redshift), which is designed to make it easier to get data from Amazon Redshift into a Pandas DataFrame and vice versa. It transfers the data frame to S3 and then to Redshift.  Of course we also used Amazon Redshift to set up the data warehouse and to perform our queries as seen above.

* Propose how often the data should be updated and why.
 
    * For the I94 Immigration Data, it depends how often the immigration authorities update the data.  USCIS and related bodies within the government need to agree on a standard of access, updating and frequency so that the information can be synced with all agencies.  Since the data is on Amazon Redshift, we can easily provide access to this data and can agree on an update schedule and method.  For the US City Demographic Data, the data will be updated when there is another survey held by the US Census Bureau.


* Write a description of how you would approach the problem differently under the following scenarios:

    * The data was increased by 100x. 
 
        * If the data was increased by 100x, pure pandas cannot handle it. We will need Spark to handle it.  The data can be split up into multiple batches which could be used to update the tables in an asynchronous mannter.
    
    * The data populates a dashboard that must be updated on a daily basis by 7am every day.

        * We can use Airflow to schedule a pipeline that run every day before 7am in this case.
    
    * The database needed to be accessed by 100+ people.

        * The more people accessing the database the more CPU resources we need to get a fast experience.  By using a distributed database we can improve our replications and partitioning to get faster query results for each user.  However, our use of Redshift has the ability to be scalable for access so our current technology stack can support that.